Libraries

In [18]:
import pandas as pd
import statsmodels.api as sm
from scipy import stats
from sklearn.linear_model import LogisticRegression

In [19]:
nba = pd.read_csv("https://raw.githubusercontent.com/noahgift/socialpowernba/master/data/nba_2017_att_val_elo_win_housing_cluster.csv")
nba.head()

,Unnamed: 0,TEAM,GMS,PCT_ATTENDANCE,WINNING_SEASON,TOTAL_ATTENDANCE_MILLIONS,VALUE_MILLIONS,ELO,CONF,COUNTY,MEDIAN_HOME_PRICE_COUNTY_MILLIONS,COUNTY_POPULATION_MILLIONS,cluster
0,0,Chicago Bulls,41,104,1,0.888882,2500,1519,East,Cook,269900.0,5.20,1
1,1,Dallas Mavericks,41,103,0,0.811366,1450,1420,West,Dallas,314990.0,2.57,1
2,2,Sacramento Kings,41,101,0,0.721928,1075,1393,West,Sacremento,343950.0,1.51,0
3,3,Miami Heat,41,100,1,0.805400,1350,1569,East,Miami-Dade,389000.0,2.71,1
4,4,Toronto Raptors,41,100,1,0.813050,1125,1600,East,York-County,390000.0,1.10,1


In [20]:
## Now we need to create dummy variables for our categorical data
## Class = pd.get_dummies(train_df['Pclass'], prefix='Class', drop_first=True)
## Sex = pd.get_dummies(train_df['Sex'], prefix='Sex', drop_first=True)
## Embarked = pd.get_dummies(train_df['Embarked'], prefix='Embarked', drop_first=True)

Conf = pd.get_dummies(nba['CONF'], prefix='CONF', drop_first=True)
nba = pd.concat([nba, CONF], axis=1)
nba.head()

,Unnamed: 0,TEAM,GMS,PCT_ATTENDANCE,WINNING_SEASON,TOTAL_ATTENDANCE_MILLIONS,VALUE_MILLIONS,ELO,CONF,COUNTY,MEDIAN_HOME_PRICE_COUNTY_MILLIONS,COUNTY_POPULATION_MILLIONS,cluster,CONF_West
0,0,Chicago Bulls,41,104,1,0.888882,2500,1519,East,Cook,269900.0,5.20,1,0
1,1,Dallas Mavericks,41,103,0,0.811366,1450,1420,West,Dallas,314990.0,2.57,1,1
2,2,Sacramento Kings,41,101,0,0.721928,1075,1393,West,Sacremento,343950.0,1.51,0,1
3,3,Miami Heat,41,100,1,0.805400,1350,1569,East,Miami-Dade,389000.0,2.71,1,0
4,4,Toronto Raptors,41,100,1,0.813050,1125,1600,East,York-County,390000.0,1.10,1,0


In [21]:
## Let's select the columns we want to leverage in our model
features = ["WINNING_SEASON","CONF_West"]
X = nba[features]
Y = nba['TOTAL_ATTENDANCE_MILLIONS']

In [22]:
## Time to execute our prediction
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
logit_model = sm.Logit(Y,X)
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.527967
         Iterations 5
                               Logit Regression Results                              
Dep. Variable:     TOTAL_ATTENDANCE_MILLIONS   No. Observations:                   30
Model:                                 Logit   Df Residuals:                       28
Method:                                  MLE   Df Model:                            1
Date:                       Sat, 13 Jan 2018   Pseudo R-squ.:                -0.08336
Time:                               08:47:16   Log-Likelihood:                -15.839
converged:                              True   LL-Null:                       -14.620
                                               LLR p-value:                     1.000
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
WINNING_SEASON     0.8850      0.625      1.

That model didn't work as evidenced by the insignificant P-values, so lets remove the one that didn't work well.

In [23]:
## Let's select the columns we want to leverage in our model
features = ["WINNING_SEASON"]
X = nba[features]
Y = nba['TOTAL_ATTENDANCE_MILLIONS']

In [24]:
## Time to execute our prediction
stats.chisqprob = lambda chisq, df: stats.chi2.sf(chisq, df)
logit_model = sm.Logit(Y,X)
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.553834
         Iterations 5
                               Logit Regression Results                              
Dep. Variable:     TOTAL_ATTENDANCE_MILLIONS   No. Observations:                   30
Model:                                 Logit   Df Residuals:                       29
Method:                                  MLE   Df Model:                            0
Date:                       Sat, 13 Jan 2018   Pseudo R-squ.:                 -0.1364
Time:                               08:47:21   Log-Likelihood:                -16.615
converged:                              True   LL-Null:                       -14.620
                                               LLR p-value:                       nan
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
WINNING_SEASON     1.1320      0.565      2.

This model tells us that a team that has a winning season is more likely to have larger attendance for their games. This intuitively makes sense because fans are more likely to want to see games where their team wins, or there is likely to be a good showing on the court.